In [1]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import re
from pathlib import Path

In [2]:
column_names = {'Emiasid': 'emiasid', 'Emiasid ': 'emiasid' , 'emiasid ': 'emiasid', 'EMIAS_ID': 'emiasid', 'МО': 'mo', 
                'МУ': 'mu', 'Логин оператора': 'operator_login', 'Дата и время звонка': 'call_date', 
                'Результат звонка': 'call_result', 'Результат звонка ': 'call_result', 'Разговор состоялся': 'talk_result', 
                'Комменатрий': 'comment', 'Дата и время переноса': 'next_call_date', 'Контактный телефон': 'phone_number', 
                'ФИО врача': 'doctor_fio', 'ФИО помощника врача': 'assistant_fio', 'Логин помощника врача': 'operator_login',
                'Зал №': 'hall_number', 'Логин naumen': 'operator_login', 'МО закрепления': 'mo', 
                'Отсутствует на рабочем месте': 'absence', 'login': 'operator_login', 'procedure_rec_status': 'group_procedure',
                'patient_group': 'group_name'}

In [3]:
today = pd.Timestamp(dt.datetime.today().date())
today_ymd = today.strftime('%Y%m%d')
today_y_m_d = today.strftime('%Y_%m_%d')

yesterday = today - dt.timedelta(days=1)
yesterday_ymd = yesterday.strftime('%Y%m%d')
yesterday_y_m_d = yesterday.strftime('%Y_%m_%d')

tomorrow = today + dt.timedelta(days=1)
tomorrow_ymd = tomorrow.strftime('%Y%m%d')
tomorrow_y_m_d = tomorrow.strftime('%Y_%m_%d')

print(today_y_m_d)

2022_09_23


In [4]:
# путь файла с текущими статусами пациентов
patients_current_statuses_path = \
os.path.join(Path.home(), r'Documents\GitHub\final_framework\merged_statuses')

patients_current_statuses_path = patients_current_statuses_path.split('\\')
patients_current_statuses_data = os.path.join('C:\\', r'\\', *patients_current_statuses_path,
                                             f'total_patients_current_statuses_merged_with_logins_{today_y_m_d}.xlsx')
print(patients_current_statuses_data)

C:\\Users\VAIO\Documents\GitHub\final_framework\merged_statuses\total_patients_current_statuses_merged_with_logins_2022_09_23.xlsx


In [5]:
before_1 = pd.read_excel('fulltask_20220920_v20.xlsx')
before_2 = pd.read_excel('fulltask_20220921_v20.xlsx')
before_3 = pd.read_excel('fulltask_20220922_v20.xlsx')

In [6]:
before_1['emiasid'] = before_1['emiasid'].astype(str).str.replace('\.0', '')
before_2['emiasid'] = before_2['emiasid'].astype(str).str.replace('\.0', '')
before_3['emiasid'] = before_3['emiasid'].astype(str).str.replace('\.0', '')

C:\Users\VAIO\AppData\Local\Temp\ipykernel_6196\24782371.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  before_1['emiasid'] = before_1['emiasid'].astype(str).str.replace('\.0', '')
C:\Users\VAIO\AppData\Local\Temp\ipykernel_6196\24782371.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  before_2['emiasid'] = before_2['emiasid'].astype(str).str.replace('\.0', '')
C:\Users\VAIO\AppData\Local\Temp\ipykernel_6196\24782371.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  before_3['emiasid'] = before_3['emiasid'].astype(str).str.replace('\.0', '')


In [237]:
# путь файла с сегодняшними 
emias_data_path = r'\\t999\Сетевой диск\pdn\data\emias_data\xlsx'
emias_data_path = emias_data_path.split('\\')
emias_data_path = os.path.join(r'\\', *emias_data_path)
print(emias_data_path)

\\t999\Сетевой диск\pdn\data\emias_data\xlsx


In [238]:
today_emias_data_path = [os.path.join(emias_data_path, filename) for filename in os.listdir(emias_data_path) if '4059' 
                         in filename and '~' not in filename and today.strftime('%Y-%m-%d') in filename][0]
print(today_emias_data_path)

\\t999\Сетевой диск\pdn\data\emias_data\xlsx\DAT_4059_выгрузка_списков_на_обзвон_помощниками_2022-09-23.xlsx


In [239]:
# чтение всех файлов по отдельности, чтобы не тратить время на повторное чтение файла, который уже прочитан

In [240]:
%%time
today_emias_data_reading = pd.read_excel(today_emias_data_path)
patients_current_statuses_reading = pd.read_excel(patients_current_statuses_data)

CPU times: total: 2min 2s
Wall time: 2min 3s


In [241]:
import gspread
from gspread_dataframe import get_as_dataframe
gc = gspread.service_account(filename='google_sheets_credentials.json') 
url = 'https://docs.google.com/spreadsheets/d/1TXjbk9tRDMCjEyyiuK5OIgT8xwlHQ2V1PX2zaVnxPXY/edit#gid=0'
sh = gc.open_by_url(url)
ws = sh.get_worksheet(0)
def search_row(ws, desired_value):
    row_number = 0
    for row in ws.get_all_values():
        row_number += 1
        for value in row:
            if value == desired_value:
                return row_number
            
skiprows = search_row(ws, 'ФИО помощника врача') - 1

assistants_presence_reading = get_as_dataframe(ws, evaluate_formulas=True, skiprows=skiprows)
assistants_presence_reading = assistants_presence_reading.loc[~(assistants_presence_reading['Зал №'] == 'Зал №'), 
    ['Зал №', 'МО закрепления', 'ФИО помощника врача', 'Логин naumen', 'Отсутствует на рабочем месте']] \
        .dropna(how='all', axis=0)


print(dt.datetime.now())
print(assistants_presence_reading.shape[0])

assistants_presence = assistants_presence_reading.rename(columns=column_names).copy()

2022-09-23 11:20:24.814453
217


In [242]:
if assistants_presence[assistants_presence.absence.isna()].operator_login.nunique() + \
assistants_presence[assistants_presence.absence == 1].operator_login.nunique() == assistants_presence.operator_login.nunique():
    print('\nВ сумме количество отсуствующих и присутствующих равны общему количеству помощников\n')
else:
    print(5 * '\nПРОВЕРЬТЕ ТАБЛИЦУ ОТСУТСТВУЮЩИХ И ПРИСУТСТВУЮЩИХ\n!!!\n')


В сумме количество отсуствующих и присутствующих равны общему количеству помощников



In [243]:
present_assistants = assistants_presence[assistants_presence.absence.isna()]
print(present_assistants.shape[0])
absent_assistants = assistants_presence[~assistants_presence.absence.isna()]
print(absent_assistants.shape[0])

163
54


In [244]:
patients_current_statuses = patients_current_statuses_reading.rename(columns=column_names).copy()

In [245]:
today_emias_data = today_emias_data_reading.rename(columns=column_names)

In [246]:
patients_current_statuses = patients_current_statuses_reading.rename(columns=column_names).copy()
patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')
patients_current_statuses['next_call_date'] = patients_current_statuses['next_call_date'] \
.apply(lambda x: pd.to_datetime(x, dayfirst=True).date())
patients_current_statuses['last_call_date'] = patients_current_statuses['last_call_date'] \
.apply(lambda x: pd.to_datetime(x, dayfirst=True).date())
patients_current_statuses['phone_number'] = patients_current_statuses['phone_number'].astype(str).str.replace('\.0', '')

C:\Users\VAIO\AppData\Local\Temp\ipykernel_6196\277195396.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')
C:\Users\VAIO\AppData\Local\Temp\ipykernel_6196\277195396.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  patients_current_statuses['phone_number'] = patients_current_statuses['phone_number'].astype(str).str.replace('\.0', '')


In [247]:
# Переведем все типы данных емиасайди, телефоны и прочие числовые данные в строки

In [248]:
today_emias_data['emiasid'] = today_emias_data['emiasid'].astype(str).str.replace('\.0', '')
today_emias_data['phone'] = today_emias_data['phone'].astype(str).str.replace('\.0', '')

C:\Users\VAIO\AppData\Local\Temp\ipykernel_6196\3971369908.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  today_emias_data['emiasid'] = today_emias_data['emiasid'].astype(str).str.replace('\.0', '')
C:\Users\VAIO\AppData\Local\Temp\ipykernel_6196\3971369908.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  today_emias_data['phone'] = today_emias_data['phone'].astype(str).str.replace('\.0', '')


In [249]:
# путь папки с раскреплением
assistant_to_doctor_distribution_path = \
r'\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path'
assistant_to_doctor_distribution_path = \
assistant_to_doctor_distribution_path.split('\\')
assistant_to_doctor_distribution_path = \
os.path.join(r'\\', *assistant_to_doctor_distribution_path)
print(assistant_to_doctor_distribution_path)

\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path


In [250]:
september_assistant_to_doctor_distribution_path = [os.path.join(assistant_to_doctor_distribution_path, filename) for filename 
                                         in os.listdir(assistant_to_doctor_distribution_path) if '~' not in filename
                                        and 'september' in filename]
september_assistant_to_doctor_distribution_path.sort(key=lambda x: os.path.getctime(x))
september_assistant_to_doctor_distribution_path = september_assistant_to_doctor_distribution_path[-1]
print(september_assistant_to_doctor_distribution_path)

\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path\september_patient_to_assistant_distribution.xlsx


In [251]:
september_assistant_to_doctor_distribution = pd.read_excel(september_assistant_to_doctor_distribution_path, 
                                    usecols=['login', 'emiasid', 'mo', 'doctor_fio'])[['emiasid', 'login', 'mo', 'doctor_fio']]

In [252]:
doctors_list = set(september_assistant_to_doctor_distribution.doctor_fio.unique())
doctors_list.add(np.NaN)

In [253]:
def find_phone_number(text):
    """Функция ищет номера телефонов в строке, фильтрует дефектные и возвращает один с приоритетом на мобильный"""
    import re
    defect_pattern = re.compile(r"""7[^94]\d{9}|0{7,}|1{7,}|2{7,}|3{7,}|4{7,}|5{7,}|6{7,}|7{7,}|8{7,}|9{7,}""")
    text = str(text)
    text = ['7' + re.sub(r'\D', '', x)[-10:] for x in re.split(r'[,;]', text)]
    text = list(filter(lambda x: len(x) == 11, text))
    text = list(filter(lambda x: False if re.search(defect_pattern, x) else True, text))
    text = sorted(text, reverse=True)
    text = ','.join(text)
    text = re.search(r'(9\d{9}|4\d{9})', text)
    if text:
        text = text.group()
    return text

today_emias_data['phone_1'] = today_emias_data['phone'].apply(find_phone_number)
today_emias_data[today_emias_data['phone_1'].isna()].shape[0]

321

In [254]:
today_emias_data.shape[0]

97159

In [255]:
exception = pd.read_excel('exception.xlsx')

In [256]:
exception = exception.rename(columns=column_names)
exception['emiasid'] = exception['emiasid'].astype(str).str.replace('\.0', '')

C:\Users\VAIO\AppData\Local\Temp\ipykernel_6196\4054622638.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  exception['emiasid'] = exception['emiasid'].astype(str).str.replace('\.0', '')


In [257]:
exception.shape[0]

372

In [258]:
patients_current_statuses.problem.unique()

array(['Нет прикрепления к МО', nan, 'Новый врач',
       'Нет прикрепления к врачу', 'Смена МО', 'Смена МО на другую нашу',
       'Отсутствует номер'], dtype=object)

In [271]:
exclude_emiases = set(before_2['emiasid'].unique()) | set(before_3['emiasid'].unique()) | set(before_1['emiasid'].unique())

In [272]:
patients_current_statuses.shape[0]

109216

In [273]:
actual_patients = patients_current_statuses[
    ~(patients_current_statuses.problem.isin({
        'Нет в сегодняшней выгрузке', 'Нет прикрепления к МО', 'Отсутствует номер', 'Смена МО', 
        'Смена МО на другую нашу'
    }))
]
actual_patients.shape[0]

108655

In [274]:
actual_patients = actual_patients[(~(actual_patients.emiasid.isin(exception.emiasid)))]

actual_patients.shape[0]

108341

In [275]:
actual_patients = actual_patients[(~(actual_patients.emiasid.isin(exclude_emiases)))]

actual_patients.shape[0]

65202

In [276]:
actual_patients = actual_patients[actual_patients.group.isin({
    11.,  2., 14., 10., 17.,  3.,  5., 19., 22., 13., 15., 12., 18., 25., 23.,  4., 16.,  9.,  6.,  7., 8.
})]

actual_patients.shape[0]

60404

In [277]:
actual_patients = actual_patients[~actual_patients.status.isin({
    'Будет в Москве в другом месяце', 'Отказ', 'Иная МО', 'Живет не в Москве', 'Патронаж', 'Снят с ДН',
    'Онкология', 'Некорректный номер', 'Госпитализирован', 'Умер', 'Пациент отрицает диагноз', 'Отсутствует номер'
})]

actual_patients.shape[0]

46833

In [278]:
patients_current_statuses.shape[0]

109216

In [279]:
actual_patients.shape[0]

46833

In [280]:
actual_patients.status.value_counts()

Перенос звонка на другой день    18617
Не дозвонились                   17879
Разговор состоялся                5481
Ни разу не звонили                2571
Перезвонить сегодня               1345
Запишется самостоятельно           891
Не нравится врач                    32
Другой врач в МО                    15
Не нравится МО                       2
Name: status, dtype: int64

In [281]:
actual_patients.operator_login.count()

45696

In [282]:
# actual_patients.loc[actual_patients.problem == 'Новый врач', 'operator_login'] = np.nan

In [283]:
actual_patients.operator_login.count()

45696

In [284]:
first_stage_patients = \
actual_patients[
    (actual_patients.status == 'Ни разу не звонили')
    &
    (actual_patients.month == 'september')
]

first_stage_patients['priority'] = 2
first_stage_patients.shape[0]

C:\Users\VAIO\AppData\Local\Temp\ipykernel_6196\3473140784.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_stage_patients['priority'] = 2


10

In [289]:
call_back_on_date_patients = \
actual_patients[
    (actual_patients.next_call_date > dt.date(2022, 9, 2))
    &
    (actual_patients.next_call_date < dt.date(2022, 9, 24))
]

call_back_on_date_patients['priority'] = 1
call_back_on_date_patients.shape[0]

C:\Users\VAIO\AppData\Local\Temp\ipykernel_6196\718716443.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  call_back_on_date_patients['priority'] = 1


9310

In [290]:
actual_patients[
    (actual_patients.month == 'august')
    &
    (actual_patients.last_call_date < dt.date(2022, 9, 2))
].shape[0]

2530

In [291]:
actual_patients.last_call_result.unique()

array(['Перенос звонка на другой день', 'Не дозвонились',
       'Разговор состоялся', 'Нет ответа', 'Перезвонить сегодня',
       'Занято', 'Сброс', 'operator_rejected', 'Информация о ДН', 'Отказ',
       nan], dtype=object)

In [292]:
august_patients = \
actual_patients[
    (actual_patients.month == 'august')
    &
    (actual_patients.last_call_result != 'Перенос звонка на другой день')
    &
    (actual_patients.last_call_date < dt.date(2022, 9, 2))
]

# august_patients = august_patients.head(800)

august_patients['priority'] = 3
august_patients.shape[0]

C:\Users\VAIO\AppData\Local\Temp\ipykernel_6196\3798363847.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  august_patients['priority'] = 3


1178

In [293]:
august_patients.last_call_date.unique()[-1]

datetime.date(2022, 8, 26)

In [294]:
tomorrow_record_patients_set = set()
for column in today_emias_data.columns[22:98:2]:
    tomorrow_record_patients_set = \
    tomorrow_record_patients_set | set(today_emias_data[today_emias_data[f'{column}'] == tomorrow_y_m_d]\
                                   .emiasid.astype(str).unique())

print(len(tomorrow_record_patients_set))

0


In [295]:
tomorrow_record_patients = patients_current_statuses[(patients_current_statuses.emiasid.isin(tomorrow_record_patients_set))
& (~(patients_current_statuses.status.isin({'Ни разу не звонили', 'Онкология', 'Смена МО', 'Отсутствует номер', 
                                           'Пациент отрицает диагноз', 'Отказ', 'Госпитализирован',
                                           'Будет в Москве в другом месяце', 'Некорректный номер', 'Патронаж',
                                           'Снят с ДН'}))) & 
(patients_current_statuses.last_call_result != 'Перенос звонка на другой день')]
print(tomorrow_record_patients.shape[0])

0


In [296]:
tomorrow_record_patients['priority'] = 4

In [297]:
tomorrow_record_patients.status.value_counts()

Series([], Name: status, dtype: int64)

In [298]:
tomorrow_record_patients['status'] = 'Напомнить'

In [299]:
second_stage_patients = \
actual_patients[
    (actual_patients.status.isin({'Не дозвонились', 'Запишется самостоятельно', 'Не нравится МО', 'Другой врач в МО',
                                 'Не нравится врач'}))
    &
    (~(actual_patients.emiasid.isin(call_back_on_date_patients.emiasid)))
#     &
#     (~(actual_patients.last_call_date.isin(
#         {
#         dt.date(2022, 9, 19)
#         }
#     )))
]

second_stage_patients['priority'] = 5

second_stage_patients.shape[0]

C:\Users\VAIO\AppData\Local\Temp\ipykernel_6196\952935718.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_stage_patients['priority'] = 5


18566

In [300]:
final_actual_patients_for_task_ = pd.concat([
    call_back_on_date_patients,
    first_stage_patients,
    august_patients,
    second_stage_patients,
    tomorrow_record_patients
]) \
.sort_values(['priority']).reset_index(drop=True)

final_actual_patients_for_task_.shape[0]

29064

In [301]:
# final_actual_patients_for_task = final_actual_patients_for_task_.drop_duplicates(subset=['phone_number'], keep='first')
# final_actual_patients_for_task.shape[0]

In [350]:
final_actual_patients_for_task = final_actual_patients_for_task_.drop_duplicates(subset=['emiasid'], keep='first')
final_actual_patients_for_task.shape[0]

28986

In [351]:
final_actual_patients_for_task = final_actual_patients_for_task.drop_duplicates(subset=['phone_number'], keep='first')
final_actual_patients_for_task.shape[0]

28968

In [352]:
final_actual_patients_for_task.head(3)

,emiasid,status,old_comment,next_call_date,operator_login,phone_number,mo,doctor_fio,calls_amount,last_call_date,last_call_result,last_talk_result,month,problem,group,group_procedure,group_name,absence,priority
0,10206309,Ни разу не звонили,"нет слотов, актив 6157",2022-09-06,NaN,9254575732,ДЦ 5,Насриддинова Мавлюдахон Тажибаевна,2,2022-08-25,Перенос звонка на другой день,NaN,august,Новый врач,11.0,надо записать,Записать на прием - Записать на исследования,0,1
1,21378076,Перенос звонка на другой день,перезвонить,2022-09-13,pomoshnik_pdn83,9032862186,ГП 2,Котляр Владислав Альбертович,1,2022-09-12,Перенос звонка на другой день,NaN,september,NaN,12.0,в будущем,Записать на прием,0,1
2,21378599,Перенос звонка на другой день,перезвонить,2022-09-23,pomoshnik_pdn19,9055428895,КДП 121,Субботина Марина Алексеевна,1,2022-09-12,Перенос звонка на другой день,NaN,september,Новый врач,13.0,выполненная запись,Записать на прием,1,1


In [353]:
final_actual_patients_for_task = \
final_actual_patients_for_task[['emiasid', 'status', 'old_comment', 'operator_login', 'mo', 'doctor_fio', 'priority', 
                                'calls_amount', 'last_call_date', 'month']]

In [354]:
final_actual_patients_for_task['emiasid'] = final_actual_patients_for_task['emiasid'].astype(str).str.replace('\.0', '')

C:\Users\VAIO\AppData\Local\Temp\ipykernel_6196\2998909243.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  final_actual_patients_for_task['emiasid'] = final_actual_patients_for_task['emiasid'].astype(str).str.replace('\.0', '')


In [355]:
initial_assistant_to_patient_distribution_in_today_task = \
final_actual_patients_for_task.copy()

In [356]:
initial_assistant_to_patient_distribution_in_today_task.old_comment.nunique()

5021

In [357]:
initial_assistant_to_patient_distribution_in_today_task.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(
    ['mo', 'emiasid']).head(20)

emiasid
mo     operator_login           
ГП 107 pomoshnik_pdn131       22
       pomoshnik_pdn130       70
       pomoshnik_pdn43       124
       pomoshnik_pdn148      140
       pomoshnik_pdn129      156
       pomoshnik_pdn185      163
       pomoshnik_pdn109      175
ГП 109 pomoshnik_pdn155       27
       pomoshnik_pdn134       79
       pomoshnik_pdn168      140
ГП 11  pomoshnik_pdn146       50
       pomoshnik_pdn32       106
       pomoshnik_pdn147      230
       pomoshnik_pdn184      413
ГП 134 pomoshnik_pdn89       155
       pomoshnik_pdn75       191
       pomoshnik_pdn7        204
       pomoshnik_pdn93       373
       pomoshnik_pdn333      413
ГП 166 pomoshnik_pdn209       33

In [358]:
set(present_assistants.operator_login.unique()) - \
(set(initial_assistant_to_patient_distribution_in_today_task.operator_login.unique()) - {np.nan})

{'pomoshnik_pdn99'}

In [359]:
task_amount = 90

In [360]:
%%time

initial_task_for_today = pd.DataFrame()
for operator_login in set(initial_assistant_to_patient_distribution_in_today_task.operator_login):
    initial_assistant_to_patient_distribution_in_today_task_for_operator_login = \
    initial_assistant_to_patient_distribution_in_today_task[initial_assistant_to_patient_distribution_in_today_task \
                                                            .operator_login
         == operator_login].sort_values(by=['priority', 'calls_amount', 'last_call_date'])
    call_back_number_for_operator_login = initial_assistant_to_patient_distribution_in_today_task_for_operator_login[
        initial_assistant_to_patient_distribution_in_today_task_for_operator_login.status.isin({
            'Перезвонить сегодня', 'Перенос звонка на другой день'})].shape[0]

    initial_task_for_today = pd.concat(
        [initial_task_for_today, 

         
         initial_assistant_to_patient_distribution_in_today_task[
             (initial_assistant_to_patient_distribution_in_today_task.operator_login==operator_login)
         ].head(task_amount),
        
       
        ]).reset_index(drop=True)

initial_task_for_today.shape[0]

CPU times: total: 1.83 s
Wall time: 1.83 s


16392

In [361]:
initial_task_for_today \
.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'])\
.sort_values(by='emiasid', ascending=True).head(60)

,,emiasid
mo,operator_login,
ГП 170,pomoshnik_pdn151,2
ГП 68,pomoshnik_pdn241,6
ГП 19,pomoshnik_pdn246,11
ГП 46,pomoshnik_pdn200,12
ГП 52,pomoshnik_pdn103,19
ГП 107,pomoshnik_pdn131,22
ГП 6,pomoshnik_pdn173,23
ГП 22,pomoshnik_pdn174,26
ГП 109,pomoshnik_pdn155,27


In [362]:
initial_task_for_today \
.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'])\
.sort_values(by='emiasid', ascending=False).head(10)

emiasid
mo    operator_login           
ГП 45 pomoshnik_pdn257       90
      pomoshnik_pdn299       90
      Помощник 9             90
ГП 5  pomoshnik_pdn239       90
ГП 52 pomoshnik_pdn101       90
      pomoshnik_pdn194       90
      pomoshnik_pdn220       90
ГП 62 pomoshnik_pdn107       90
      Помощник 4             90
ГП 64 pomoshnik_pdn105       90

In [363]:
rename_mo = \
{'ГБУЗ Г МОСКВЫ ГП № 170 ДЗМ': 'ГП 170', ' ГП 6 ДЗМ': 'ГП 6', 'ГБУЗ КДЦ 6 ДЗМ':'КДЦ 6', 
'ГБУЗ "ГП № 134 ДЗМ"': 'ГП 134', 'КДЦ 4': 'КДЦ 4', 'ГБУЗ ДКЦ 1 ДЗМ': 'ДКЦ 1', 'ГБУЗ ГП 23 ДЗМ': 'ГП 23',
'ГБУЗ ГП 109 ДЗМ': 'ГП 109', 'ГБУЗ ГП 11 ДЗМ': 'ГП 11', 'ГБУЗ ГП 64 ДЗМ': 'ГП 64', 'ГБУЗ ГП 191 ДЗМ': 'ГП 191', 
'ГБУЗ ГП 220 ДЗМ': 'ГП 220', 'ГБУЗ "ГП № 218 ДЗМ"': 'ГП 218',  'ГБУЗ ГП 107 ДЗМ': 'ГП 107', 
'ГБУЗ "ГП № 22 ДЗМ"': 'ГП 22', 'ГБУЗ ДЦ 5 ДЗМ': 'ДЦ 5', 'ГБУЗ ГП 210 ДЗМ': 'ГП 210', 'ГБУЗ ГП 214 ДЗМ': 'ГП 214', 
'ГБУЗ ГП 2 ДЗМ': 'ГП 2', 'ГБУЗ КДП 121 ДЗМ': 'КДП 121', 'ГБУЗ ГП 66 ДЗМ': 'ГП 66',  'ГБУЗ ДЦ 3 ДЗМ ': 'ДЦ 3',
'ГБУЗ ГП 36 ДЗМ': 'ГП 36',  'ГБУЗ ГП 62 ДЗМ ': 'ГП 62', 
'ГБУЗ "ГП № 52 ДЗМ"': 'ГП 52', 'ГБУЗ "ГП 67 ДЗМ"': 'ГП 67', 'ГБУЗ ГП 68 ДЗМ': 'ГП 68', 'ГБУЗ ГП 175 ДЗМ': 'ГП 175',
'ГБУЗ ГП № 209 ДЗМ': 'ГП 209',  'ГБУЗ ГП 212 ДЗМ': 'ГП 212', 'ГБУЗ ГП 19 ДЗМ': 'ГП 19', 
'ГБУЗ ГП №166 ДЗМ': 'ГП 166', 'ГБУЗ ГП 9 ДЗМ': 'ГП 9', 'ГБУЗ ГП 69 ДЗМ': 'ГП 69',
'ГБУЗ "ГП № 3 ДЗМ"': 'ГП 3', 'ГБУЗ ГП 5 ДЗМ': 'ГП 5', 'ГБУЗ ГП 46 ДЗМ': 'ГП 46', 'ГБУЗ ГП №8 ДЗМ': 'ГП 8', 
'ГБУЗ "ГП № 45 ДЗМ"': 'ГП 45'}

In [364]:
mo_list = set(rename_mo.values())

In [365]:
initial_task_for_today_present_assistants = \
initial_task_for_today[~initial_task_for_today.operator_login.isin(absent_assistants.operator_login)]

In [366]:
initial_task_for_today[initial_task_for_today.operator_login.isin(absent_assistants.operator_login)].shape[0]

4468

In [367]:
initial_task_for_today_present_assistants.shape[0]

11924

In [368]:
initial_task_for_today_present_assistants.groupby('operator_login').agg({'emiasid': 'count'}).sort_values('emiasid', ascending=False)

,emiasid
operator_login,
pomoshnik_pdn101,90
pomoshnik_pdn278,90
pomoshnik_pdn43,90
pomoshnik_pdn41,90
pomoshnik_pdn40,90
...,...
pomoshnik_pdn131,22
pomoshnik_pdn103,19
pomoshnik_pdn200,12


In [369]:
initial_task_for_today_absent_assistants =  \
initial_task_for_today[initial_task_for_today.operator_login.isin(absent_assistants.operator_login)] \
[['emiasid', 'status', 'old_comment', 'mo', 'priority', 'month']]

In [370]:
initial_task_for_today_absent_assistants.columns

Index(['emiasid', 'status', 'old_comment', 'mo', 'priority', 'month'], dtype='object')

In [371]:
new_doctor_patients = actual_patients[actual_patients.problem == 'Новый врач']

In [372]:
new_doctor_patients['priority'] = 6

C:\Users\VAIO\AppData\Local\Temp\ipykernel_6196\1491646637.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_doctor_patients['priority'] = 6


In [373]:
new_doctor_patients = new_doctor_patients[['emiasid', 'status', 'old_comment', 'mo', 'priority', 'month']]
new_doctor_patients.shape[0]

25852

In [374]:
additional_task_for_present_assistants = \
pd.concat([
#     new_doctor_patients,
    initial_task_for_today_absent_assistants
]).reset_index(drop=True)
additional_task_for_present_assistants.shape[0]

4468

In [375]:
additional_task_for_present_assistants_by_mo = \
additional_task_for_present_assistants.groupby(['mo']).agg({'emiasid': 'count'}).sort_values(by=['mo', 'emiasid'])

In [376]:
distribution_count = initial_task_for_today_present_assistants.groupby(['mo']) \
.agg({'emiasid': 'count', 'operator_login': 'nunique'}) \
.sort_values(by=['mo', 'emiasid'])

In [377]:
distribution_count['total_delta'] = distribution_count['operator_login'] * task_amount - distribution_count['emiasid']

In [378]:
distribution_count = distribution_count[distribution_count.total_delta > 0]

In [379]:
additional_task_for_present_assistants

,emiasid,status,old_comment,mo,priority,month
0,21815441,Перенос звонка на другой день,"нет слотов, номер актива 14044",ГП 36,1,september
1,21578361,Перенос звонка на другой день,"нет слотов, номер актива 14122",ГП 36,1,september
2,20551396,Перенос звонка на другой день,"перезвонить, записать на исследования",ГП 36,1,september
3,20808150,Перезвонить сегодня,NaN,ГП 36,1,september
4,21066907,Перенос звонка на другой день,"перезвонить,контроль записи на гастроскопию",ГП 36,1,september
...,...,...,...,...,...,...
4463,115434391,Ни разу не звонили,"перезвонить, болеет",ГП 214,1,august
4464,21986878,Ни разу не звонили,"перезвонить, записать к терапевту",ГП 214,1,august
4465,16045919,Ни разу не звонили,"перезвонить, записан к терапевту",ГП 214,1,august
4466,536675566,Ни разу не звонили,"перезвонить, напомнить про запись",ГП 214,1,august


In [380]:
final_task_predistribution = \
pd.concat([initial_task_for_today_present_assistants, additional_task_for_present_assistants]).reset_index(drop=True)

In [381]:
empty_login_patients_amount_before = final_task_predistribution[final_task_predistribution.operator_login.isna()].shape[0]
empty_login_patients_amount_before

4468

In [382]:
%%time

for mo in distribution_count.index:
    
    while final_task_predistribution[final_task_predistribution.mo == mo].groupby('operator_login') \
    .agg({'emiasid': 'count'}).min()['emiasid'] < task_amount and \
    final_task_predistribution[(final_task_predistribution.operator_login.isna()) & (final_task_predistribution.mo == mo)] \
    .shape[0] > 0:
        
        miminal_in_mo_assistant_emiasid_amount = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('operator_login').agg({'emiasid': 'count'}).min()['emiasid']

        minimal_in_mo_emiasid_amount_login = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('operator_login').agg({'emiasid': 'count'}).sort_values(by='emiasid').index[0]
        
        
        empty_login_in_mo_index = final_task_predistribution[
            (final_task_predistribution.operator_login.isna()) & (final_task_predistribution.mo == mo)
        ].index[0]
        
        final_task_predistribution = \
        final_task_predistribution.sort_values(by=['priority', 'calls_amount', 'last_call_date'])

        final_task_predistribution.loc[empty_login_in_mo_index, 'operator_login'] = minimal_in_mo_emiasid_amount_login


CPU times: total: 24.9 s
Wall time: 25 s


In [401]:
empty_login_patients_amount_after = final_task_predistribution[final_task_predistribution.operator_login.isna()].shape[0]
empty_login_patients_amount_after

3013

In [402]:
check_table = distribution_count.merge(additional_task_for_present_assistants_by_mo,left_index=True, right_index=True) \
[['total_delta', 'emiasid_y']]

In [403]:
check_table.loc[check_table['total_delta'] > check_table['emiasid_y'], 'real_addition'] = \
check_table.loc[check_table['total_delta'] > check_table['emiasid_y'], 'emiasid_y']

check_table.loc[check_table['total_delta'] <= check_table['emiasid_y'], 'real_addition'] = \
check_table.loc[check_table['total_delta'] <= check_table['emiasid_y'], 'total_delta']

In [404]:
real_change_of_emty_login_patients_amount = check_table.real_addition.sum()
real_change_of_emty_login_patients_amount

1455.0

In [405]:
empty_login_patients_amount_before - empty_login_patients_amount_after == real_change_of_emty_login_patients_amount

True

In [406]:
final_task_distribution = final_task_predistribution[~final_task_predistribution.operator_login.isna()] \
.copy().reset_index(drop=True)

In [407]:
final_task_distribution.shape[0]

13379

In [408]:
final_task_distribution.status.value_counts()

Не дозвонились                   5892
Перенос звонка на другой день    5217
Перезвонить сегодня              1025
Ни разу не звонили                816
Запишется самостоятельно          403
Разговор состоялся                 10
Не нравится врач                   10
Другой врач в МО                    5
Не нравится МО                      1
Name: status, dtype: int64

In [409]:
final_task_distribution.shape[0]

13379

In [410]:
final_task_distribution.groupby('operator_login').agg({'emiasid': 'count'}).sort_values(by=['emiasid'], ascending=True).head(50)

,emiasid
operator_login,
pomoshnik_pdn151,2
pomoshnik_pdn241,6
pomoshnik_pdn200,12
pomoshnik_pdn174,26
pomoshnik_pdn66,38
pomoshnik_pdn242,38
pomoshnik_pdn235,38
pomoshnik_pdn199,39
pomoshnik_pdn240,42


In [411]:
final_task_distribution['emiasid'] = final_task_distribution['emiasid']

In [412]:
final_task_distribution = final_task_distribution.drop_duplicates(subset=['emiasid'], keep='first')
final_task_distribution.shape[0]

13379

In [413]:
final_task_distribution.status.value_counts()

Не дозвонились                   5892
Перенос звонка на другой день    5217
Перезвонить сегодня              1025
Ни разу не звонили                816
Запишется самостоятельно          403
Разговор состоялся                 10
Не нравится врач                   10
Другой врач в МО                    5
Не нравится МО                      1
Name: status, dtype: int64

In [414]:
final_task_distribution = final_task_distribution.drop_duplicates(subset=['emiasid'], keep='first')
final_task_distribution.shape[0]

13379

In [415]:
final_task_distribution.operator_login.value_counts()

pomoshnik_pdn131    90
pomoshnik_pdn134    90
pomoshnik_pdn111    90
pomoshnik_pdn113    90
pomoshnik_pdn208    90
                    ..
pomoshnik_pdn235    38
pomoshnik_pdn174    26
pomoshnik_pdn200    12
pomoshnik_pdn241     6
pomoshnik_pdn151     2
Name: operator_login, Length: 162, dtype: int64

In [417]:
final_task_distribution[final_task_distribution.month == 'august']\
.to_excel(rf'august_patients_data_{today_y_m_d}.xlsx', index=False)

In [418]:
final_task_distribution[final_task_distribution.month == 'september']\
.to_excel(rf'september_patients_data_{today_y_m_d}.xlsx', index=False)

In [419]:
final_task_distribution.to_excel(rf'full_task_for_today_{today_y_m_d}.xlsx', index=False)